# Importing Libraries and Datasets

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Training Dataset

In [ ]:
dataset = pd.read_csv('train.csv')
X = dataset.iloc[:, 1:-6].values
y = dataset.iloc[:, -6:-4].values

## Testing Dataset

In [ ]:
Test_dataset = pd.read_csv('test.csv')
X_submission = Test_dataset.iloc[:, 1:].values

# Feature Scaling

## Training Dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

## Test Dataset

In [ ]:
X_submission = sc.transform(X_submission)

# Training models and predicting

## CATBOOST

### For EC1

In [ ]:
from catboost import CatBoostRegressor
regressor_cat1 = CatBoostRegressor(learning_rate=0.007573490922788006,
                               depth=6,
                               subsample=0.8147194080189883,
                               colsample_bylevel=0.5311715736954817,
                               min_data_in_leaf=56)
regressor_cat1.fit(X, y[:, 0])

y_pred_cat1 = regressor_cat1.predict(X)

### For EC2

In [ ]:
regressor_cat2 = CatBoostRegressor(learning_rate=0.01632700367902254,
                               depth=3,
                               subsample=0.6001420520780691,
                               colsample_bylevel=0.5074737933892164,
                               min_data_in_leaf=75)
regressor_cat2.fit(X, y[:, 1])

y_pred_cat2 = regressor_cat2.predict(X)

## XGBoost

### For EC1

In [ ]:
from xgboost import XGBRegressor
regressor_xgb1 = XGBRegressor(max_depth=2,
                          learning_rate = 0.06888600413221835,
                          n_estimators=201,
                          min_child_weight=3,
                          gamma=0.0356537935597181,
                          subsample=0.919214627352363,
                          colsample_bytree=0.6479551640312314,
                          reg_alpha=0.4805888998558167,
                          reg_lambda=0.2234388007912691)
regressor_xgb1.fit(X, y[:, 0])

y_pred_xgb1 = regressor_xgb1.predict(X)

### For EC2

In [ ]:
regressor_xgb2 = XGBRegressor(max_depth=4,
                          learning_rate =0.04847250666447861,
                          n_estimators=279,
                          min_child_weight=3,
                          gamma=0.847297033527792,
                          subsample=0.7434346764669405,
                          colsample_bytree=0.09679280536046835,
                          reg_alpha=0.894699651732954,
                          reg_lambda=0.6729634389162842)
regressor_xgb2.fit(X, y[:, 1])

y_pred_xgb2 = regressor_xgb2.predict(X)

# Creating arrays for ANNs inputs

## For EC1

In [ ]:
y_in1 = np.concatenate((y_pred_cat1.reshape(len(y_pred_cat1), 1),
                        y_pred_xgb1.reshape(len(y_pred_xgb1), 1)),
                        1)

In [ ]:
print(y_in1)

## For EC2

In [ ]:
y_in2 = np.concatenate((y_pred_cat2.reshape(len(y_pred_cat2), 1),
                        y_pred_xgb2.reshape(len(y_pred_xgb2), 1)),
                        1)

In [ ]:
print(y_in2)

# ANN Building

## Building

### ANN 1

In [ ]:
ann1 = tf.keras.models.Sequential()

ann1.add(tf.keras.layers.Dense(units=5, activation='relu'))

ann1.add(tf.keras.layers.Dense(units=5, activation='relu'))

ann1.add(tf.keras.layers.Dense(units=1))

### ANN 2

In [ ]:
ann2 = tf.keras.models.Sequential()

ann2.add(tf.keras.layers.Dense(units=5, activation='relu'))

ann2.add(tf.keras.layers.Dense(units=5, activation='relu'))

ann2.add(tf.keras.layers.Dense(units=1))

In [ ]:
print(y_in1.shape)

## Compiling and Training

### ANN 1

In [ ]:
ann1.compile(optimizer = 'adam', loss = 'mean_squared_error')
ann1.fit(y_in1, y[:,0], batch_size = 32, epochs = 120)

### ANN 2

In [ ]:
ann2.compile(optimizer = 'adam', loss = 'mean_squared_error')
ann2.fit(y_in2, y[:, 1], batch_size = 32, epochs = 120)

# Predicting Test Set Data

In [ ]:
y_submission_cat1 = regressor_cat1.predict(X_submission)
y_submission_cat2 = regressor_cat2.predict(X_submission)
y_submission_xgb1 = regressor_xgb1.predict(X_submission)
y_submission_xgb2 = regressor_xgb2.predict(X_submission)

In [ ]:
y_submission1 = np.concatenate((y_submission_cat1.reshape(len(y_submission_cat1), 1),
                                y_submission_xgb1.reshape(len(y_submission_xgb1), 1)),
                                1)

y_submission2 = np.concatenate((y_submission_cat2.reshape(len(y_submission_cat2), 1),
                                y_submission_xgb2.reshape(len(y_submission_xgb2), 1)),
                                1)

In [ ]:
y_pred1 = ann1.predict(y_submission1)
y_pred2 = ann2.predict(y_submission2)

In [ ]:
print(y_pred1)

In [ ]:
print(y_pred2)

# Creating Submission File

In [ ]:
submission = pd.DataFrame({'id':Test_dataset['id'],'EC1': y_pred1.flatten(), 'EC2': y_pred2.flatten()})
submission.to_csv('submission.csv',index=False)